#Predicciones de Ventas

##Librerías

In [56]:
import pandas as pd
import plotly.express as px

from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

##Datos

###Carga

Establecemos una conexión con Drive para exportar los archivos csv

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


Cargamos los archivos

In [3]:
channel = pd.read_csv('/content/drive/MyDrive/Data/channel.csv')
country = pd.read_csv('/content/drive/MyDrive/Data/country.csv')
sells = pd.read_csv('/content/drive/MyDrive/Data/sells.csv')
validate = pd.read_csv('/content/drive/MyDrive/Data/validate.csv')

###Análisis

In [ ]:
channel.shape

(2, 2)

In [ ]:
channel.isna().sum()

channel_id    0
channel       0
dtype: int64

In [ ]:
channel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   channel_id  2 non-null      int64 
 1   channel     2 non-null      object
dtypes: int64(1), object(1)
memory usage: 160.0+ bytes


In [ ]:
channel.sample()

,channel_id,channel
0,1,ecommerce


In [ ]:
country.shape

(7, 2)

In [ ]:
country.isna().sum()

country_id    0
country       0
dtype: int64

In [ ]:
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country_id  7 non-null      int64 
 1   country     7 non-null      object
dtypes: int64(1), object(1)
memory usage: 240.0+ bytes


In [ ]:
country.sample()

,country_id,country
4,5,Guatemala


In [ ]:
sells.shape

(11690, 4)

In [ ]:
sells.isna().sum()

date          0
sales         0
country_id    0
channel_id    0
dtype: int64

In [ ]:
sells.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11690 entries, 0 to 11689
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        11690 non-null  object
 1   sales       11690 non-null  int64 
 2   country_id  11690 non-null  int64 
 3   channel_id  11690 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 365.4+ KB


In [ ]:
sells.sample()

,date,sales,country_id,channel_id
6850,2023-03-02,1441,7,2


In [ ]:
validate.shape

(210, 4)

In [ ]:
validate.isna().sum()

date          0
sales         0
country_id    0
channel_id    0
dtype: int64

In [ ]:
validate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        210 non-null    object
 1   sales       210 non-null    int64 
 2   country_id  210 non-null    int64 
 3   channel_id  210 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.7+ KB


In [ ]:
validate.sample()

,date,sales,country_id,channel_id
138,2023-06-13,11535,3,2


##Transformaciones

In [4]:
df = sells.copy()

In [5]:
df['datetime'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S%z')
df['datetime'] = df['datetime'].dt.date

Unimos los conjuntos de datos a través de los campos de id 

In [6]:
df = pd.concat([df.set_index('channel_id'), channel.set_index('channel_id')], axis=1)
df = pd.concat([df.set_index('country_id'), country.set_index('country_id')], axis=1)

In [7]:
df.reset_index(inplace=True)

In [8]:
df

,country_id,date,sales,datetime,channel,country
0,5,2022-11-07,19658,2022-11-07,in-store,Guatemala
1,5,2022-03-21,15405,2022-03-21,in-store,Guatemala
2,5,2022-09-26,220676,2022-09-26,ecommerce,Guatemala
3,4,2022-01-23,3102,2022-01-23,in-store,Canada
4,4,2022-08-12,82700,2022-08-12,ecommerce,Canada
...,...,...,...,...,...,...
11685,5,2023-03-29,114824,2023-03-29,ecommerce,Guatemala
11686,6,2021-11-06,9939,2021-11-06,in-store,Belice
11687,4,2021-04-06,43901,2021-04-06,ecommerce,Canada
11688,4,2022-04-06,2048,2022-04-06,in-store,Canada


Observamos el listado de paises

In [9]:
df.country.unique()

array(['Guatemala', 'Canada', 'USA', 'Argentina', 'México', 'Brasil',
       'Belice'], dtype=object)

Instanciamos una variable LabelEncoder

In [12]:
le = LabelEncoder()

Aplicamos el encoder a la columna country

In [14]:
le.fit(df['country'])

LabelEncoder()

In [15]:
df['le_country'] = le.transform(df['country'])

Realizamos lo mismo para la columna channel

In [17]:
le.fit(df['channel'])
df['le_channel'] = le.transform(df['channel'])

In [18]:
df

,country_id,date,sales,datetime,channel,country,le_country,le_channel
0,5,2022-11-07,19658,2022-11-07,in-store,Guatemala,4,1
1,5,2022-03-21,15405,2022-03-21,in-store,Guatemala,4,1
2,5,2022-09-26,220676,2022-09-26,ecommerce,Guatemala,4,0
3,4,2022-01-23,3102,2022-01-23,in-store,Canada,3,1
4,4,2022-08-12,82700,2022-08-12,ecommerce,Canada,3,0
...,...,...,...,...,...,...,...,...
11685,5,2023-03-29,114824,2023-03-29,ecommerce,Guatemala,4,0
11686,6,2021-11-06,9939,2021-11-06,in-store,Belice,1,1
11687,4,2021-04-06,43901,2021-04-06,ecommerce,Canada,3,0
11688,4,2022-04-06,2048,2022-04-06,in-store,Canada,3,1


Creamos un nuevo dataframe con las agrupaciones por fecha y país 

In [47]:
df_tot = df.groupby(['date', 'country']).agg({'sales': 'sum'}).reset_index()
df_tot

,date,country,sales
0,2021-02-15,Argentina,21818
1,2021-02-15,Belice,29342
2,2021-02-15,Brasil,26201
3,2021-02-15,Canada,7756
4,2021-02-15,Guatemala,35599
...,...,...,...
5840,2023-05-30,Brasil,163086
5841,2023-05-30,Canada,117213
5842,2023-05-30,Guatemala,206434
5843,2023-05-30,México,319146


Creamos tabla pivote con relación fecha - país

In [48]:
df_pivot = df_tot.pivot(index='date', columns='country', values='sales')
df_pivot

country,Argentina,Belice,Brasil,Canada,Guatemala,México,USA
date,,,,,,,
2021-02-15,21818,29342,26201,7756,35599,42650,20964
2021-02-16,39760,54372,51463,35379,66471,92590,43300
2021-02-17,31670,45062,41892,19515,56826,70939,34583
2021-02-18,60188,91824,84719,44110,99623,176830,72821
2021-02-19,62344,93143,88446,40663,102802,180348,74389
...,...,...,...,...,...,...,...
2023-05-26,137159,176215,204845,108812,244795,430726,173630
2023-05-27,140400,178299,217319,118535,252070,448827,173354
2023-05-28,146225,179900,212422,109324,249759,455928,179594


Agregamos una columna con la suma de ventas por día

In [49]:
df_pivot['TotalSales'] = df_pivot.apply(lambda row: row['Argentina'] 
                                         + row['Belice'] + row['Brasil']
                                         + row['Canada'] + row['Guatemala'] 
                                         + row['México'] + row['USA'], axis=1)
df_pivot.reset_index(inplace=True)
df_pivot

country,date,Argentina,Belice,Brasil,Canada,Guatemala,México,USA,TotalSales
0,2021-02-15,21818,29342,26201,7756,35599,42650,20964,184330
1,2021-02-16,39760,54372,51463,35379,66471,92590,43300,383335
2,2021-02-17,31670,45062,41892,19515,56826,70939,34583,300487
3,2021-02-18,60188,91824,84719,44110,99623,176830,72821,630115
4,2021-02-19,62344,93143,88446,40663,102802,180348,74389,642135
...,...,...,...,...,...,...,...,...,...
830,2023-05-26,137159,176215,204845,108812,244795,430726,173630,1476182
831,2023-05-27,140400,178299,217319,118535,252070,448827,173354,1528804
832,2023-05-28,146225,179900,212422,109324,249759,455928,179594,1533152
833,2023-05-29,151804,196147,233533,122967,261316,480423,194327,1640517


Creamos una visualización de los datos

In [55]:
fig = px.line(df_pivot, x='date', y='TotalSales', title='Total de Ventas')

fig.update_xaxes(title='Fecha')
fig.update_yaxes(title='Ventas')

fig.show()

##Predicción

In [120]:
df = df_pivot.copy()

Convertimos la columna date a datetime

In [121]:
df['date'] = pd.to_datetime(df['date'])

Determinamos el día después al último registro

In [122]:
fecha_inicio = df['date'].max() + timedelta(1)

Creamos un conjunto con los próximos 15 días para su predicción

In [123]:
fechas_pred = pd.date_range(fecha_inicio, periods=15, freq='D')

###Modelado (Regresión Lineal)

#####Ventas Próximos 15 Días

Definimos las variables X y y para el modelo

In [113]:
X = df['date'].dt.dayofyear.values.reshape(-1, 1)
y = df['TotalSales'].values

Entrenamos el modelo de Regresión Lineal

In [114]:
linreg = LinearRegression()
linreg.fit(X, y)

LinearRegression()

Realizamos la predicción de ventas para los próximos 15 días

In [115]:
X_pred = fechas_pred.dayofyear.values.reshape(-1, 1)
ventas_pred = linreg.predict(X_pred)

In [116]:
df_pred = pd.DataFrame({
    'Fecha': fechas_pred,
    'TotalSales': ventas_pred
})

In [117]:
df_pred.TotalSales = df_pred.TotalSales.apply(lambda x: int(x))

In [118]:
df_pred

,Fecha,TotalSales
0,2023-05-31,1060547
1,2023-06-01,1060799
2,2023-06-02,1061051
3,2023-06-03,1061303
4,2023-06-04,1061554
5,2023-06-05,1061806
6,2023-06-06,1062058
7,2023-06-07,1062310
8,2023-06-08,1062562
9,2023-06-09,1062813


#####México

Obtenemos el promedio de véntas histórico

In [124]:
promedio_ventas = df['México'].mean()

Definimos las variables X y y para el modelo

In [125]:
X = df['date'].dt.dayofyear.values.reshape(-1, 1)
y = df['México'].values

Entrenamos el modelo de regresión lineal

In [126]:
linreg = LinearRegression()
linreg.fit(X, y)

LinearRegression()

Predecimos la cantidad de entregas de México

In [127]:
X_pred = fechas_pred.dayofyear.values.reshape(-1, 1)
ventas_pred = linreg.predict(X_pred)

Evaluamos la condición; Ventas Pred > Mean

In [128]:
results = ventas_pred > promedio_ventas

In [129]:
df_pred = pd.DataFrame({
    'Fecha': fechas_pred,
    'Entregas': ventas_pred,
    'Results': results
})

In [130]:
df_pred

,Fecha,Entregas,Results
0,2023-05-31,296562.706742,False
1,2023-06-01,296610.802211,False
2,2023-06-02,296658.897680,False
3,2023-06-03,296706.993149,False
4,2023-06-04,296755.088618,False
5,2023-06-05,296803.184087,False
6,2023-06-06,296851.279556,False
7,2023-06-07,296899.375025,False
8,2023-06-08,296947.470494,False
9,2023-06-09,296995.565963,False
